# <center> Домашнее задание №1. Решение
## <center>  Разведочный анализ данных с помощью Pandas


**В этом задании нужно с помощью Pandas ответить на несколько вопросов о наборе данных [Adult](https://archive.ics.uci.edu/ml/datasets/Adult). (Скачивать данные не нужно — они уже находятся в репозитории.)**

Уникальные значения признаков (подробности см. по ссылке выше):
- `age`: непрерывный признак.
- `workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- `fnlwgt`: непрерывный признак.
- `education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- `education-num`: непрерывный признак.
- `marital-status`: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- `occupation`: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- `relationship`: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- `race`: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- `sex`: Female, Male.
- `capital-gain`: непрерывный признак.
- `capital-loss`: непрерывный признак.
- `hours-per-week`: непрерывный признак.
- `native-country`: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.   
- `salary`: >50K,<=50K

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('../data/adult.data.csv')
data.head()

**1. Сколько мужчин и женщин (признак *sex*) представлено в этом датасете?** 

In [ ]:
data['sex'].value_counts()

**2. Каков средний возраст женщин (признак *age*)?**

In [ ]:
data.loc[data['sex'] == 'Female', 'age'].mean()

**3. Какова доля граждан Германии (признак *native-country*)?**

In [ ]:
float((data['native-country'] == 'Germany').sum()) / data.shape[0]

**4–5. Каковы среднее значение и стандартное отклонение возраста тех, кто получает больше 50K в год (признак *salary*), и тех, кто получает не более 50K в год? **

In [ ]:
ages1 = data.loc[data['salary'] == '>50K', 'age']
ages2 = data.loc[data['salary'] == '<=50K', 'age']
print("The average age of the rich: {0} +- {1} years, poor - {2} +- {3} years.".format(
    round(ages1.mean()), round(ages1.std(), 1),
    round(ages2.mean()), round(ages2.std(), 1)))

**6. Верно ли, что люди, зарабатывающие больше 50K, имеют как минимум школьное образование? (признак *education* — Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters или *Doctorate*)**

In [ ]:
data.loc[data['salary'] == '>50K', 'education'].unique() # No

**7. Выведите статистики возраста для каждой расы (признак *race*) и каждого пола. Используйте *groupby()* и *describe()*. Найдите максимальный возраст мужчин расы *Amer-Indian-Eskimo*.**

In [ ]:
for (race, sex), sub_df in data.groupby(['race', 'sex']):
    print("Race: {0}, sex: {1}".format(race, sex))
    print(sub_df['age'].describe())

**8. У кого доля хорошо зарабатывающих (>50K) выше: среди женатых или среди неженатых мужчин (признак *marital-status*)? Считайте женатыми тех, у кого *marital-status* начинается с *Married* (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считайте холостыми.**

In [ ]:
data.loc[(data['sex'] == 'Male') &
     (data['marital-status'].isin(['Never-married', 
                                   'Separated', 
                                   'Divorced',
                                   'Widowed'])), 'salary'].value_counts()

In [ ]:
data.loc[(data['sex'] == 'Male') &
     (data['marital-status'].str.startswith('Married')), 'salary'].value_counts()

In [ ]:
data['marital-status'].value_counts()

Быть женатым — неплохо :)

**9. Какое максимальное количество часов в неделю (*hours-per-week*) кто‑то работает? Сколько людей работают столько часов и каков процент хорошо зарабатывающих среди них?**

In [ ]:
max_load = data['hours-per-week'].max()
print("Max time - {0} hours./week.".format(max_load))

num_workaholics = data[data['hours-per-week'] == max_load].shape[0]
print("Total number of such hard workers {0}".format(num_workaholics))

rich_share = float(data[(data['hours-per-week'] == max_load)
                 & (data['salary'] == '>50K')].shape[0]) / num_workaholics
print("Percentage of rich among them {0}%".format(int(100 * rich_share)))

**10. Посчитайте среднее количество рабочих часов (*hours-per-week*) для мало зарабатывающих и хорошо зарабатывающих (*salary*) в каждой стране (*native-country*).**

Простой способ:

In [ ]:
for (country, salary), sub_df in data.groupby(['native-country', 'salary']):
    print(country, salary, round(sub_df['hours-per-week'].mean(), 2))

Элегантный способ:

In [ ]:
pd.crosstab(data['native-country'], data['salary'], 
           values=data['hours-per-week'], aggfunc=np.mean).T